# Setup

In [ ]:
# Image from Drive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --quiet -U --no-cache-dir gdown --pre
!pip install --quiet nlp-id
!pip install --quiet emoji
!pip install --quiet imblearn
!pip install --quiet gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 71.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 69.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from nlp_id.stopword import StopWord
from nlp_id.postag import PosTag
from nlp_id.lemmatizer import Lemmatizer

from imblearn.over_sampling import ADASYN 

from xgboost.sklearn import XGBClassifier

import emoji
import string
import re
import numpy as np
import pandas as pd

import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')

import seaborn as sns # untuk visualisasi
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

# potongan kode di bawah adalah agar gambar graph yang 
# dihasilkan mempunyai kualitas yang cukup baik
sns.set(rc={"figure.dpi":100, 'savefig.dpi':300})
sns.set_context('notebook')
sns.set_style("ticks")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Tokenization & Emoticon Processor

regex_str = []
regex_str.append(r'<[^>]+>') # HTML tags
regex_str.append(r'(?:@[\w_]+)') # @-mentions
regex_str.append(r'(?:&[\w_]+)')
regex_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
regex_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
regex_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers
regex_str.append(r"(?:[a-z][a-z'\-_]+[a-z])") # words with - and '
regex_str.append(r'(?:[\w_]+)') # other words
regex_str.append(r'(?:\S)') # anything else

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)

def tokenize(sentence):
    return [emoji.demojize(term) for term in tokens_re.findall(sentence)]

In [ ]:
# Stopwords Removal & Lemmatizing

lemmatizer = Lemmatizer()
stopword = StopWord()

punctuation = list(string.punctuation)
stops = stopword.get_stopword() + punctuation + ['rt', 'via', '…','•','“', 'user', 'url',
                                                  'xf0', 'x9f', 'x98', 'x84', 'xf0', 'x9f', 'x98', 'x84', 'xf0', 'x9f', 
                                                  'x98', 'x84', 'xf', 'xe2', 'x80', 'x94', 'x82', 'xa4', 'x9d', 'xc2',
                                                  'x86', 'x85', 'x81', 'xa6', 'xa3', 'xad', 'x91', 'x81', 'xe3', 'x99',
                                                  'x', 'a', 'n', 'b',
                                                  '1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

def remove_stops_and_lemmatize(tokenized_sentence):
  processed_words = []
  for word in tokenized_sentence:
    if word.lower() in kamus_alay.Alay.values:
      row = kamus_alay[kamus_alay['Alay'] == word.lower()]
      word = row['Tidak Alay'].values[0]
    if word.lower() not in stops:
      processed_words.append(lemmatizer.lemmatize(word))
  return processed_words

# Load Test Data

In [ ]:
abusive_lexicon = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/abusive.csv')
kamus_alay = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/kamusalay.csv', encoding='latin-1')
dataset = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/re_dataset.csv', encoding='latin-1')

test_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/test_data.csv', encoding='latin-1')
potensi_bias = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/potensi_bias.csv', encoding='latin-1')
replacement_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/bias_replacement.csv', encoding='latin-1')

In [ ]:
kamus_alay

Alay                         Tidak Alay
0      anakjakartaasikasik           anak jakarta asyik asyik
1             pakcikdahtua                  pak cik sudah tua
2           pakcikmudalagi                  pak cik muda lagi
3              t3tapjokowi                       tetap jokowi
4                       3x                          tiga kali
...                    ...                                ...
15162            mendikbud  menteri pendidikan dan kebudayaan
15163               mendag                menteri perdagangan
15164              menaker               menteri tenaga kerja
15165             memetwit                            mentwit
15166             megangin                           memegang

[15167 rows x 2 columns]

In [ ]:
replacement_data

Unnamed: 0            0              r1            r2             r3  \
0          0.0       jokowi        soekarno        habibi         kelvin   
1          1.0        islam           bahai        shinto       tenrikyo   
2          2.0    indonesia          jepang         korea          india   
3          3.0      prabowo          arthur       filbert          dimas   
4          4.0         cina         vietnam         rusia        meksiko   
5          5.0         ahok            doni         baraz         zaidan   
6          6.0     presiden         menteri      pemimpin         bupati   
7          7.0          kpk              ky           kpu          polri   
8          8.0     gubernur          bupati      walikota        pegawai   
9          9.0          sby        soeharto      megawati         gusdur   
10        10.0        anies            adit        fathan           anto   
11        11.0          dpr             mpr           dpd             mk   
12        12.0       muslim        agnostik      jainisme           sikh   
13        13.0        sandi          farrel         zidan          yazid   
14        14.0  anies-sandi  soekarno-hatta   hamzah-agum  amien-siswono   
15        15.0     rohingya          melayu        mulato          metis   
16        16.0      kristen       helenisme  neopaganisme       mahayana   
17        17.0       buddha          deisme     zoroaster         shinto   
18        18.0      myanmar        filipina    afganistan         brasil   
19        19.0        china         vietnam         rusia        meksiko   
20         NaN          NaN             NaN           NaN            NaN   
21         NaN          NaN             NaN           NaN            NaN   
22         NaN          NaN               -           NaN            NaN   

                    r4               r5  
0                 arie           nathan  
1             animisme       shamanisme  
2            australia         perancis  
3               khariz             gabe  
4              amerika          inggris  
5                usman              adi  
6             walikota          pegawai  
7                   bi              tni  
8             pemimpin          menteri  
9                angga             sean  
10             sulthan             reza  
11                  ma              bpk  
12              mormon         henoteis  
13               rifqy            ditya  
14  wiranto-salahuddin  megawati-hasyim  
15              serani         tionghoa  
16             taoisme       politeisme  
17       konfusianisme       chondoisme  
18              jerman         malaysia  
19             amerika          inggris  
20                 NaN              NaN  
21                 NaN              NaN  
22                 NaN              NaN

# Replace Potential Bias with Replacement Words

In [ ]:
ds_test = test_data.copy()
ds_test

Unnamed: 0.1  Unnamed: 0  prev_id  \
0                0           0      298   
1                1           1     2634   
2                2           2     2404   
3                3           3     5128   
4                4           4     6099   
...            ...         ...      ...   
3945          3945        3945      336   
3946          3946        3946     3521   
3947          3947        3947     2307   
3948          3948        3948     8971   
3949          3949        3949     5065   

                                                  Tweet  HS  Abusive  \
0     USER Ooo camtu.. Biasalah kena seimbang.. Ada ...   0        0   
1     Presiden Minta Sistem Perizinan Berusaha Terin...   0        0   
2     ð???ð???ð???tak berambisi? Mimin bisa aja judu...   0        0   
3     USER Kemaren gubernur pilihan umat impor bawan...   0        0   
4     4. Apakah pernah mengalami kendala dalam menga...   0        0   
...                                                 ...  ..      ...   
3945  Doi tuh native speaker ya.\n\nMother language-...   1        1   
3946  RT USER Kasian ya kacung tetap kacung anies sandi   1        1   
3947  USER kacrut , rank 10 tapi mainnya gk cerdas a...   1        1   
3948  USER Klau kmu di banding AR jauhlah.koen menun...   1        1   
3949  Mana para babi ahok yg ngejek onta, biar di in...   1        1   

      HS_Individual  HS_Group  HS_Religion  HS_Race  HS_Physical  HS_Gender  \
0                 0         0            0        0            0          0   
1                 0         0            0        0            0          0   
2                 0         0            0        0            0          0   
3                 0         0            0        0            0          0   
4                 0         0            0        0            0          0   
...             ...       ...          ...      ...          ...        ...   
3945              1         0            0        0            0          0   
3946              0         1            0        0            0          0   
3947              1         0            0        0            0          0   
3948              1         0            0        0            0          0   
3949              0         1            0        0            0          0   

      HS_Other  HS_Weak  HS_Moderate  HS_Strong  
0            0        0            0          0  
1            0        0            0          0  
2            0        0            0          0  
3            0        0            0          0  
4            0        0            0          0  
...        ...      ...          ...        ...  
3945         1        1            0          0  
3946         1        0            1          0  
3947         1        1            0          0  
3948         1        1            0          0  
3949         1        0            1          0  

[3950 rows x 16 columns]

In [ ]:
ds_test = ds_test[0:0]
ds_test

Empty DataFrame
Columns: [Unnamed: 0.1, Unnamed: 0, prev_id, Tweet, HS, Abusive, HS_Individual, HS_Group, HS_Religion, HS_Race, HS_Physical, HS_Gender, HS_Other, HS_Weak, HS_Moderate, HS_Strong]
Index: []

- 0: 2692
- 1: 760 
- 2: 349
- 3: 101
- 4: 29
- 5: 13
- 6: 4
- 7: 1
- 8: 1



In [ ]:
def replace_bias_alay(sentence):
  sen = sentence
  tokenized_sentence = tokenize(sentence)
  for word in tokenized_sentence:
    if word.lower() in kamus_alay.Alay.values:
      row = kamus_alay[kamus_alay['Alay'] == word.lower()]
      rep = row['Tidak Alay'].values[0]
      if rep in replacement_data["0"].values:
        sen = sen.replace(word, rep)
  return sen

In [ ]:
import random
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

for i, row in test_data.iterrows():
  new_row = row
  words = remove_stops_and_lemmatize(tokenize(row["Tweet"]))
  bias_list = []
  pool = ['r1', 'r2', 'r3', 'r4', 'r5']

  sentence = replace_bias_alay(row.Tweet)

  tweet_1 = sentence.lower()
  tweet_2 = sentence.lower()
  tweet_3 = sentence.lower()
  tweet_4 = sentence.lower()
  tweet_5 = sentence.lower()

  copy_row_1 = row.copy()
  copy_row_2 = row.copy()
  copy_row_3 = row.copy()
  copy_row_4 = row.copy()
  copy_row_5 = row.copy()

  for word in words:
    if word.lower() in replacement_data["0"].values:
      bias_list.append(word)

  for bias in bias_list:
    bias_row = replacement_data[replacement_data['0'] == bias.lower()]
    random.shuffle(pool)

    tweet_1 = tweet_1.replace(bias, str(bias_row[pool[0]].values[0]))
    tweet_2 = tweet_2.replace(bias, str(bias_row[pool[1]].values[0]))
    tweet_3 = tweet_3.replace(bias, str(bias_row[pool[2]].values[0]))
    tweet_4 = tweet_4.replace(bias, str(bias_row[pool[3]].values[0]))
    tweet_5 = tweet_5.replace(bias, str(bias_row[pool[4]].values[0]))

  copy_row_1.Tweet = tweet_1
  copy_row_2.Tweet = tweet_2
  copy_row_3.Tweet = tweet_3
  copy_row_4.Tweet = tweet_4
  copy_row_5.Tweet = tweet_5

  ds_test = ds_test.append(copy_row_1)
  ds_test = ds_test.append(copy_row_2)
  ds_test = ds_test.append(copy_row_3)
  ds_test = ds_test.append(copy_row_4)
  ds_test = ds_test.append(copy_row_5)

# Store Dataset

In [ ]:
ds_test


Unnamed: 0.1  Unnamed: 0  prev_id  \
0                0           0      298   
0                0           0      298   
0                0           0      298   
0                0           0      298   
0                0           0      298   
...            ...         ...      ...   
3949          3949        3949     5065   
3949          3949        3949     5065   
3949          3949        3949     5065   
3949          3949        3949     5065   
3949          3949        3949     5065   

                                                  Tweet  HS  Abusive  \
0     user ooo camtu.. biasalah kena seimbang.. ada ...   0        0   
0     user ooo camtu.. biasalah kena seimbang.. ada ...   0        0   
0     user ooo camtu.. biasalah kena seimbang.. ada ...   0        0   
0     user ooo camtu.. biasalah kena seimbang.. ada ...   0        0   
0     user ooo camtu.. biasalah kena seimbang.. ada ...   0        0   
...                                                 ...  ..      ...   
3949  mana para babi adi yg ngejek onta, biar di inj...   1        1   
3949  mana para babi doni yg ngejek onta, biar di in...   1        1   
3949  mana para babi zaidan yg ngejek onta, biar di ...   1        1   
3949  mana para babi baraz yg ngejek onta, biar di i...   1        1   
3949  mana para babi usman yg ngejek onta, biar di i...   1        1   

      HS_Individual  HS_Group  HS_Religion  HS_Race  HS_Physical  HS_Gender  \
0                 0         0            0        0            0          0   
0                 0         0            0        0            0          0   
0                 0         0            0        0            0          0   
0                 0         0            0        0            0          0   
0                 0         0            0        0            0          0   
...             ...       ...          ...      ...          ...        ...   
3949              0         1            0        0            0          0   
3949              0         1            0        0            0          0   
3949              0         1            0        0            0          0   
3949              0         1            0        0            0          0   
3949              0         1            0        0            0          0   

      HS_Other  HS_Weak  HS_Moderate  HS_Strong  
0            0        0            0          0  
0            0        0            0          0  
0            0        0            0          0  
0            0        0            0          0  
0            0        0            0          0  
...        ...      ...          ...        ...  
3949         1        0            1          0  
3949         1        0            1          0  
3949         1        0            1          0  
3949         1        0            1          0  
3949         1        0            1          0  

[19750 rows x 16 columns]

In [ ]:
from google.colab import files
ds_test.to_csv('synthetic_test_data_5x.csv', encoding = 'latin-1') 
files.download('synthetic_test_data_5x.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Sanity Check

In [ ]:
for i, row in ds_test.iterrows():
  new_row = row
  words = remove_stops_and_lemmatize(tokenize(row["Tweet"]))
  bias = []
  pool = ['r1', 'r2', 'r3', 'r4', 'r5']

  for word in words:
    if word.lower() in replacement_data["0"].values:
      bias.append(word)

  if len(bias) > 0:
    print(i, bias)